## Settings

In [1]:
import sklearn
print(sklearn.__version__)

1.6.0


In [3]:
import os
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler

os.getcwd()
version = 'PlusMinus3_v3'
raw_data_path = 'basic_info_df_modified_PlusMinus3.csv'
basic_data_path = f'{version}_basic_data.csv'
normalized_data_path = f'{version}_normalized_data.csv'

raw_data = pd.read_csv(raw_data_path)

In [4]:
version = 'Basic_data_agg_v1'
basic_data_path = f'{version}.csv'
normalized_data_path = f'{version}.csv'

raw_data_list = []
for percent in [2, 3]:
    raw_data_path = f'basic_info_df_modified_PlusMinus{percent}.csv'
    df = pd.read_csv(raw_data_path)
    df["예가범위"] = percent
    raw_data_list.append(df)
    
raw_data = pd.concat(raw_data_list, ignore_index=True)

# 1. 필요한 함수 모음, 실행

In [28]:
# 원하는 컬럼 순서 정의
desired_columns = [
    '공고번호',
    '기초금액',
    'A값',
    '투찰률(%)',
    '참여업체수',
    '예가범위',
    '공고구분표시',
    '공고제목',
    '발주처(수요기관)',
    '지역제한'
]

# DataFrame 재구성
def reorganize_dataframe(df, desired_columns):
    """
    DataFrame의 컬럼을 원하는 순서로 재배열하고, 
    숫자 컬럼들은 그대로 유지합니다.
    """
    # 12번째 이후의 컬럼들 가져오기
    remaining_columns = [col for col in df.columns if col not in desired_columns and col.count('_') == 1 and len(col.split('_')[0]) == 3 and len(col.split('_')[1]) == 3]
    
    # 원하는 순서의 컬럼과 나머지 컬럼들 합치기
    final_columns = desired_columns + remaining_columns
    # 새로운 순서로 DataFrame 재구성
    reorganized_df = df[final_columns]
    return reorganized_df


def normalize_and_transform_data(df):
    """
    DataFrame의 특정 컬럼들을 변환하고 정규화합니다.
    
    변환:
    - 기초금액 -> log 변환 후 정규화
    - A값 -> 기초금액으로 나눈 후 정규화
    - 투찰률 -> 정규화
    - 참여업체수 -> log 변환 후 정규화
    """
    import numpy as np
    from sklearn.preprocessing import StandardScaler

    # 결과 DataFrame 복사
    result_df = df.copy()
    
    # 스케일러 초기화
    scaler = StandardScaler()
    
    # 1. 기초금액 변환
    result_df['log_기초금액'] = np.log1p(df['기초금액'].astype(float))
    result_df['log_기초금액_norm'] = scaler.fit_transform(result_df[['log_기초금액']])
    result_df['기초금액_norm'] = scaler.fit_transform(result_df[['기초금액']])
    
    # 2. A값/기초금액 변환
    result_df['A값/기초금액_norm'] = df['A값'].astype(float) / (1+df['기초금액'].astype(float))
    result_df['A값/기초금액_norm'] = scaler.fit_transform(result_df[['A값/기초금액_norm']])
    
    # 3. 투찰률 정규화
    result_df['투찰률_norm'] = scaler.fit_transform(df[['투찰률(%)']])
    
    # 4. 참여업체수 변환
    result_df['log_참여업체수'] = np.log1p(1+ df['참여업체수'].astype(float))
    result_df['log_참여업체수_norm'] = scaler.fit_transform(result_df[['log_참여업체수']])
    result_df['참여업체수_norm'] = scaler.fit_transform(result_df[['참여업체수']])

    # 5. 참여업체수/기초금액 변환
    num_colums = [col for col in result_df.columns if col.count('_') == 1 and len(col.split('_')[0]) == 3 and len(col.split('_')[1]) == 3]
    for col in num_colums:
       result_df[col] = result_df[col] / result_df['참여업체수']
    
    # 나머지 컬럼들은 이름 유지
    # 나머지 컬럼들은 이름 유지
    keep_columns = [
        '공고번호',  # 식별자이므로 첫 번째로
        '기초금액',
        'log_기초금액',
        'log_기초금액_norm',
        '기초금액_norm',
        'A값',
        'A값/기초금액_norm',
        '투찰률(%)',
        '투찰률_norm',
        '참여업체수',
        'log_참여업체수',
        'log_참여업체수_norm',
        '예가범위',
        '공고구분표시',
        '공고제목',
        '발주처(수요기관)',
        '지역제한',
    ]
    
    # 최종 컬럼 순서 설정
    final_columns = keep_columns + num_colums
    result_df = result_df[final_columns]

    return result_df

In [8]:
# DataFrame 재구성 실행
raw_data_reorganized = reorganize_dataframe(raw_data, desired_columns)

# normalize 실행
transformed_df = normalize_and_transform_data(raw_data_reorganized)

# CSV 파일로 저장
transformed_df.to_csv(normalized_data_path, index=False)

# Pipeline 함수 정의

In [29]:
def process_pipeline(df, desired_columns):
    # reorganize data
    organized_df = reorganize_dataframe(df, desired_columns)
    # normalize data
    normalized_df = normalize_and_transform_data(organized_df)
    return organized_df, normalized_df
organized_df, normalized_df = process_pipeline(raw_data, desired_columns)

# save data
# organized_data_path = 'Basic_data_agg_v4.csv'
# normalized_data_path = 'Modified_Basic_data_agg_v4.csv'
# # organized_df.to_csv(basic_data_path, index=False)
# organized_df.to_csv(organized_data_path, index=False)
# normalized_df.to_csv(normalized_data_path, index=False)
# shuffled_df = shuffle_specific_columns(normalized_df)
# shuffled_df.to_csv(f'Shuffled_Basic_data_agg_v4.csv', index=False)

In [26]:
organized_df.columns[:30]

Index(['공고번호', '기초금액', 'A값', '투찰률(%)', '참여업체수', '예가범위', '공고구분표시', '공고제목',
       '발주처(수요기관)', '지역제한', '010_001', '010_002', '010_003', '010_004',
       '010_005', '010_006', '010_007', '010_008', '010_009', '010_010',
       '020_001', '020_002', '020_003', '020_004', '020_005', '020_006',
       '020_007', '020_008', '020_009', '020_010'],
      dtype='object')

In [30]:
Basic_X = organized_df.iloc[:,:10]
Basic_X.columns
Basic_Y = organized_df.iloc[:,10:]
Basic_Y.set_index(Basic_X['공고번호'], inplace=True)
print(Basic_Y.columns[:10])

Modified_X = normalized_df.iloc[:,:17]
Modified_X.columns
Modified_Y = normalized_df.iloc[:,17:]
Modified_Y.set_index(Modified_X['공고번호'], inplace=True)
Modified_Y.columns[:10]

Index(['010_001', '010_002', '010_003', '010_004', '010_005', '010_006',
       '010_007', '010_008', '010_009', '010_010'],
      dtype='object')


Index(['010_001', '010_002', '010_003', '010_004', '010_005', '010_006',
       '010_007', '010_008', '010_009', '010_010'],
      dtype='object')

In [33]:
Basic_X.to_csv('Basic_X.csv', index=False)
Basic_Y.to_csv('Basic_Y.csv', index=True)
Modified_X.to_csv('Modified_X.csv', index=False)
Modified_Y.to_csv('Modified_Y.csv', index=True)


In [10]:
# Maked randomized
def shuffle_specific_columns(df):
    """
    지정된 컬럼들만 독립적으로 셔플합니다.
    
    Parameters:
    -----------
    df : DataFrame
        원본 데이터프레임
    
    Returns:
    --------
    DataFrame
        지정된 컬럼들만 셔플된 데이터프레임
    """
    import numpy as np
    
    columns_to_shuffle = [
        '기초금액',
        'log_기초금액',
        'log_기초금액_norm',
        '기초금액_norm',
        'A값',
        'A값/기초금액_norm',
        '투찰률_norm',
        '참여업체수',
        'log_참여업체수',
        'log_참여업체수_norm',
        '예가범위',
        '공고구분표시',
        '발주처(수요기관)',
        '지역제한',
        '정답사정률(%)'
    ]
    
    shuffled_df = df.copy()
    
    # 지정된 컬럼만 셔플
    for col in columns_to_shuffle:
        if col in df.columns:  # 해당 컬럼이 존재하는 경우에만 셔플
            shuffled_df[col] = np.random.permutation(df[col].values)

    return shuffled_df